In [ ]:
# pip install nltk

#Installing Libraries


In [17]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from numpy.linalg import norm
import pandas as pd
from fractions import Fraction
import re
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# s1 = "sunshine state enjoy sunshine"
# s2 = "brown fox jump high, brown fox run"
# s3 = "sunshine state fox run fast"
s1 =  "sunshine state enjoy sunshine".lower()
s2 = "brown fox jump high brown fox run".lower()
s3 = "sunshine state fox run fast".lower()

s2=re.sub(r"[,]","",s2)
corpus = [s1,s2,s3]
df1 = pd.DataFrame(corpus)
df1

,0
0,sunshine state enjoy sunshine
1,brown fox jump high brown fox run
2,sunshine state fox run fast


In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

# Vocabolary

In [ ]:
features = vectorizer.get_feature_names_out()
features

array(['brown', 'enjoy', 'fast', 'fox', 'high', 'jump', 'run', 'state',
       'sunshine'], dtype=object)

# Bag of Words

In [ ]:
print(X.toarray())

[[0 1 0 0 0 0 0 1 2]
 [2 0 0 2 1 1 1 0 0]
 [0 0 1 1 0 0 1 1 1]]


In [ ]:
df2 = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=["s1","s2","s3"])
df2

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
s1,0,1,0,0,0,0,0,1,2
s2,2,0,0,2,1,1,1,0,0
s3,0,0,1,1,0,0,1,1,1


# TF

In [ ]:
df2.loc["s1"] = [Fraction(i,len(s1.split())) for i in df2.loc["s1"]]
df2.loc["s2"] = [Fraction(i,len(s2.split())) for i in df2.loc["s2"]]
df2.loc["s3"] = [Fraction(i,len(s3.split())) for i in df2.loc["s3"]]

In [ ]:
df2

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
s1,0,1/4,0,0,0,0,0,1/4,1/2
s2,2/7,0,0,2/7,1/7,1/7,1/7,0,0
s3,0,0,1/5,1/5,0,0,1/5,1/5,1/5


# IDF

In [ ]:
total_docs = len(df2)
idf = dict()

for i in range(len(features)):
  value = 0
  if features[i] in s1.split():
    value+=1
  if features[i] in s2.split():
    value+=1
  if features[i] in s3.split():
    value+=1
  idf[features[i]]= math.log10(Fraction(total_docs,value))

In [ ]:
pd.DataFrame(idf,index=["IDF values"])

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
IDF values,0.477121,0.477121,0.477121,0.176091,0.477121,0.477121,0.176091,0.176091,0.176091


# TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

In [ ]:
tfidf_vector = tfidf_vectorizer.fit_transform(corpus)

In [ ]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=["s1","s2","s3"], columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,brown,enjoy,fast,fox,high,jump,run,state,sunshine
s1,0.000000,0.50689,0.000000,0.000000,0.000000,0.000000,0.000000,0.385503,0.771006
s2,0.670703,0.00000,0.000000,0.510087,0.335352,0.335352,0.255044,0.000000,0.000000
s3,0.000000,0.00000,0.549351,0.417796,0.000000,0.000000,0.417796,0.417796,0.417796


# Cosine Similarity

In [27]:
S1 = np.array([0,1,0,0,0,0,0,1,2])
S3 = np.array([0,0,1,1,0,0,1,1,1])

In [28]:
sim = np.dot(S1,S3)/(norm(S1)*norm(S3))
print("Cosine Similarity of S1 and S3 = ", sim)

Cosine Similarity of S1 and S3 =  0.5477225575051661
